# 07: LLM-Powered Knowledge Graph Extraction with Graphistry

**llamatelemetry v0.1.0** | Kaggle 2× Tesla T4 (30GB Total VRAM)

---

## 🎯 Objective

This notebook demonstrates **LLM-powered knowledge graph extraction** using llamatelemetry v0.1.0 with graphistry[ai] visualization:

### Architecture:
```
┌────────────────────────────────────────────────────────┐
│  GPU 0: llama-server (LLM)                             │
│         Extract entities & relationships from text     │
│         tensor_split="1.0,0.0"                         │
└────────────────────────────────────────────────────────┘
                          ↓
                    Knowledge Graph
                          ↓
┌────────────────────────────────────────────────────────┐
│  GPU 1: RAPIDS + Graphistry                            │
│         cuGraph analytics + interactive visualization  │
│         CUDA_VISIBLE_DEVICES="1"                       │
└────────────────────────────────────────────────────────┘
```

### Workflow:
1. Start llama-server on GPU 0
2. Extract entities and relationships from text using LLM
3. Build knowledge graph with cuGraph on GPU 1
4. Analyze with GPU-accelerated algorithms
5. Visualize with Graphistry

---

**Previous:** [06-split-gpu-graphistry](06-split-gpu-graphistry-llamatelemetry-v0.1.0.ipynb)  
**Next:** [08-document-network-analysis](08-document-network-analysis-graphistry-llamatelemetry-v0-1-0.ipynb)

## Step 0: Add Graphistry Secrets in Kaggle

Go to **Kaggle → Settings → Add-ons → Secrets** and add:
- `Graphistry_Personal_Key_ID`
- `Graphistry_Personal_Secret_Key`

## Step 1: Verify Dual GPU Environment

Computes graph metrics (centrality, communities, PageRank) on GPU 1 to identify key entities and relationship patterns in the knowledge graph.

Verifies dual T4 GPU environment for knowledge graph extraction workflow combining LLM entity extraction with GPU-accelerated graph analytics and visualization.

In [1]:
import subprocess
import os

print("="*70)
print("🔍 SPLIT-GPU ENVIRONMENT CHECK")
print("="*70)

result = subprocess.run(
    ["nvidia-smi", "--query-gpu=index,name,memory.total,memory.free", "--format=csv,noheader"],
    capture_output=True, text=True
)

gpus = result.stdout.strip().split('\n')
print(f"\n📊 Detected {len(gpus)} GPU(s):")
for gpu in gpus:
    print(f"   {gpu}")

if len(gpus) >= 2:
    print("\n✅ Dual T4 ready for split-GPU operation!")
    print("   GPU 0 → llama-server (LLM for entity extraction)")
    print("   GPU 1 → RAPIDS/Graphistry (graph analytics)")
else:
    print("\n⚠️ Need 2 GPUs for split operation")

🔍 SPLIT-GPU ENVIRONMENT CHECK

📊 Detected 2 GPU(s):
   0, Tesla T4, 15360 MiB, 14913 MiB
   1, Tesla T4, 15360 MiB, 14913 MiB

✅ Dual T4 ready for split-GPU operation!
   GPU 0 → llama-server (LLM for entity extraction)
   GPU 1 → RAPIDS/Graphistry (graph analytics)


## Step 2: Install Dependencies

Installs llamatelemetry, RAPIDS (cuGraph/cuDF), and Graphistry dependencies for building knowledge graphs from unstructured text using LLM extraction and GPU visualization.

Installs llamatelemetry v0.1.0, RAPIDS cuGraph for GPU graph operations, and Graphistry for interactive knowledge graph visualization.

In [2]:
%%time
print("📦 Installing dependencies...")


!pip install -q huggingface-hub sseclient-py

# Install llamatelemetry v0.1.0
!pip install -q --no-cache-dir git+https://github.com/llamatelemetry/llamatelemetry.git@v0.1.0

# Install cuGraph (matching Kaggle RAPIDS 25.6.0)
!pip install -q --extra-index-url=https://pypi.nvidia.com "cugraph-cu12==25.6.*"

# Install Graphistry
!pip install -q "graphistry[ai]"

# Verify
import llamatelemetry
print(f"\n✅ llamatelemetry {llamatelemetry.__version__} installed")

try:
    import cudf, cugraph
    print(f"✅ cuDF {cudf.__version__}")
    print(f"✅ cuGraph {cugraph.__version__}")
except ImportError as e:
    print(f"⚠️ RAPIDS: {e}")

try:
    import graphistry
    print(f"✅ Graphistry {graphistry.__version__}")
except ImportError as e:
    print(f"⚠️ Graphistry: {e}")

📦 Installing dependencies...
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 41.1 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.1/42.1 MB 46.6 MB/s eta 0:00:00:00:0100:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-adk 1.22.1 requires google-cloud-bigquery-storage>=2.0.0, which is not installed.
bigframes 2.26.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
datasets 4.4.2 requires pyarrow>=21.0.0, but you have pyarrow 19.0.1 which is incompatible.
bigframes 2.26.0 requires rich<14,>=12.4.4, but you have rich 14.2.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 332.3/332.3 kB 8.7 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━


🎯 llamatelemetry v0.1.0 First-Time Setup - Kaggle 2× T4 Multi-GPU

🎮 GPU Detected: Tesla T4 (Compute 7.5)
  ✅ Tesla T4 detected - Perfect for llamatelemetry v0.1.0!
🌐 Platform: Colab

📦 Downloading Kaggle 2× T4 binaries (~961 MB)...
    Features: FlashAttention + Tensor Cores + Multi-GPU tensor-split

➡️  Attempt 1: HuggingFace (llamatelemetry-v0.1.0-cuda12-kaggle-t4x2.tar.gz)
📥 Downloading v0.1.0 from HuggingFace Hub...
   Repo: waqasm86/llamatelemetry-binaries
   File: v0.1.0/llamatelemetry-v0.1.0-cuda12-kaggle-t4x2.tar.gz


/usr/local/lib/python3.12/dist-packages/huggingface_hub/file_download.py:979: UserWarning: `local_dir_use_symlinks` parameter is deprecated and will be ignored. The process to download files to a local folder has been updated and do not rely on symlinks anymore. You only need to pass a destination folder as`local_dir`.
For more details, check out https://huggingface.co/docs/huggingface_hub/main/en/guides/download#download-files-to-local-folder.
  warnings.warn(


v0.1.0/llamatelemetry-v0.1.0-cuda12-kagg(…):   0%|          | 0.00/1.40G [00:00<?, ?B/s]

🔐 Verifying SHA256 checksum...
   ✅ Checksum verified
📦 Extracting llamatelemetry-v0.1.0-cuda12-kaggle-t4x2.tar.gz...
Found 21 files in archive
Extracted 21 files to /root/.cache/llamatelemetry/extract_0.1.0
✅ Extraction complete!
  Found bin/ and lib/ under /root/.cache/llamatelemetry/extract_0.1.0/llamatelemetry-v0.1.0-cuda12-kaggle-t4x2
  Copied 13 binaries to /usr/local/lib/python3.12/dist-packages/llamatelemetry/binaries/cuda12
  Copied 2 libraries to /usr/local/lib/python3.12/dist-packages/llamatelemetry/lib
✅ Binaries installed successfully!


✅ llamatelemetry 0.1.0 installed
✅ cuDF 25.06.00
✅ cuGraph 25.06.00
✅ Graphistry 0.50.6
CPU times: user 59.6 s, sys: 15.2 s, total: 1min 14s
Wall time: 1min 47s


## Step 3: Download GGUF Model

Downloads a language model optimized for named entity recognition and relationship extraction to power the knowledge graph construction pipeline.

Registers Graphistry API credentials from Kaggle secrets to enable cloud-based interactive knowledge graph visualization and sharing.

In [3]:
%%time
from huggingface_hub import hf_hub_download
import os

MODEL_REPO = "bartowski/Llama-3.2-3B-Instruct-GGUF"
MODEL_FILE = "Llama-3.2-3B-Instruct-Q4_K_M.gguf"

print(f"📥 Downloading {MODEL_FILE}...")

model_path = hf_hub_download(
    repo_id=MODEL_REPO,
    filename=MODEL_FILE,
    local_dir="/kaggle/working/models"
)

size_gb = os.path.getsize(model_path) / (1024**3)
print(f"\n✅ Model downloaded: {model_path}")
print(f"   Size: {size_gb:.2f} GB")

📥 Downloading Llama-3.2-3B-Instruct-Q4_K_M.gguf...


Llama-3.2-3B-Instruct-Q4_K_M.gguf:   0%|          | 0.00/2.02G [00:00<?, ?B/s]


✅ Model downloaded: /kaggle/working/models/Llama-3.2-3B-Instruct-Q4_K_M.gguf
   Size: 1.88 GB
CPU times: user 4.32 s, sys: 8.12 s, total: 12.4 s
Wall time: 27.8 s


## Step 4: Start llama-server on GPU 0 Only

Starts llama-server on GPU 0 for LLM-powered entity and relationship extraction while reserving GPU 1 for graph analytics and visualization.

Downloads language model optimized for named entity recognition and relationship extraction from unstructured text documents.

In [4]:
from llamatelemetry.server import ServerManager

print("="*70)
print("🚀 STARTING LLAMA-SERVER ON GPU 0")
print("="*70)

print("\n📋 Configuration:")
print("   GPU 0: 100% (llama-server for entity extraction)")
print("   GPU 1: 0% (reserved for RAPIDS/Graphistry)")

server = ServerManager()
server.start_server(
    model_path=model_path,
    host="127.0.0.1",
    port=8090,
    gpu_layers=99,
    tensor_split="1.0,0.0",
    ctx_size=4096,
)

if server.check_server_health():
    print("\n✅ llama-server running on GPU 0!")
else:
    print("\n❌ Server failed to start")

🚀 STARTING LLAMA-SERVER ON GPU 0

📋 Configuration:
   GPU 0: 100% (llama-server for entity extraction)
   GPU 1: 0% (reserved for RAPIDS/Graphistry)
GPU Check:
  Platform: kaggle
  GPU: Tesla T4
  Compute Capability: 7.5
  Status: ✓ Compatible
Starting llama-server...
  Executable: /usr/local/lib/python3.12/dist-packages/llamatelemetry/binaries/cuda12/llama-server
  Model: Llama-3.2-3B-Instruct-Q4_K_M.gguf
  GPU Layers: 99
  Context Size: 4096
  Server URL: http://127.0.0.1:8090
Waiting for server to be ready...... ✓ Ready in 3.0s

✅ llama-server running on GPU 0!


## Step 5: Extract Knowledge Graph from Text using LLM

Uses the LLM to extract named entities (people, organizations, locations) and their relationships from unstructured text documents.

Starts llama-server on GPU 0 with tensor-split (1.0,0.0) for entity extraction while reserving GPU 1 for graph analytics and visualization.

In [5]:
from llamatelemetry.api.client import LlamaCppClient
import json

print("="*70)
print("🧠 LLM-POWERED KNOWLEDGE EXTRACTION")
print("="*70)

client = LlamaCppClient(base_url="http://127.0.0.1:8090")

# Sample documents about GPU computing
documents = [
    "NVIDIA develops CUDA, a parallel computing platform for GPUs. CUDA enables GPU acceleration for deep learning.",
    "llama.cpp is a library for LLM inference. It supports GGUF quantization for efficient model deployment.",
    "Graphistry provides GPU-accelerated graph visualization. It integrates with RAPIDS cuGraph for analytics.",
    "RAPIDS cuDF and cuGraph accelerate data science on GPUs. They use CUDA for parallel processing.",
    "PyTorch supports distributed training with NCCL. NCCL provides multi-GPU communication primitives.",
]

# Extract entities and relationships
all_relationships = []

for i, doc in enumerate(documents, 1):
    print(f"\n📄 Document {i}:")
    print(f"   {doc[:60]}...")
    
    prompt = f"""Extract entities and relationships from this text. Format as JSON with 'relationships' array where each item has 'source', 'target', 'type'.

Text: {doc}

Output only valid JSON:"""
    
    response = client.chat.create(
        messages=[{"role": "user", "content": prompt}],
        max_tokens=300,
        temperature=0.3,
    )
    
    # Parse LLM response
    try:
        content = response.choices[0].message.content
        # Extract JSON from response
        json_start = content.find('{')
        json_end = content.rfind('}') + 1
        if json_start != -1 and json_end > json_start:
            json_str = content[json_start:json_end]
            data = json.loads(json_str)
            if 'relationships' in data:
                all_relationships.extend(data['relationships'])
                print(f"   ✅ Extracted {len(data['relationships'])} relationships")
    except Exception as e:
        print(f"   ⚠️ Parse error: {e}")

print(f"\n✅ Total relationships extracted: {len(all_relationships)}")
print(f"\n📊 Sample relationships:")
for rel in all_relationships[:5]:
    print(f"   {rel['source']} --[{rel['type']}]--> {rel['target']}")

🧠 LLM-POWERED KNOWLEDGE EXTRACTION

📄 Document 1:
   NVIDIA develops CUDA, a parallel computing platform for GPUs...
   ✅ Extracted 2 relationships

📄 Document 2:
   llama.cpp is a library for LLM inference. It supports GGUF q...
   ✅ Extracted 2 relationships

📄 Document 3:
   Graphistry provides GPU-accelerated graph visualization. It ...
   ✅ Extracted 2 relationships

📄 Document 4:
   RAPIDS cuDF and cuGraph accelerate data science on GPUs. The...
   ✅ Extracted 3 relationships

📄 Document 5:
   PyTorch supports distributed training with NCCL. NCCL provid...
   ✅ Extracted 2 relationships

✅ Total relationships extracted: 11

📊 Sample relationships:
   NVIDIA --[develops]--> CUDA
   CUDA --[enables]--> GPU
   llama.cpp --[supports]--> LLM
   llama.cpp --[supports]--> GGUF
   Graphistry --[uses]--> GPU


## Step 6: Initialize RAPIDS on GPU 1

Verifies GPU memory split confirming LLM is loaded on GPU 0 while GPU 1 remains free for RAPIDS graph processing.

In [6]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

print("="*70)
print("🔥 INITIALIZING RAPIDS ON GPU 1")
print("="*70)

import cudf
import cupy as cp

print(f"\n📊 RAPIDS GPU Info:")
device = cp.cuda.Device(0)  # Device 0 in filtered view = actual GPU 1
print(f"   Device: {device.id} (filtered view)")
print(f"   Actual GPU: 1 (Tesla T4)")

print(f"\n✅ RAPIDS initialized on GPU 1")

🔥 INITIALIZING RAPIDS ON GPU 1

📊 RAPIDS GPU Info:
   Device: 0 (filtered view)
   Actual GPU: 1 (Tesla T4)

✅ RAPIDS initialized on GPU 1


## Step 7: Build Knowledge Graph with cuGraph

Initializes RAPIDS on GPU 1 by setting CUDA_VISIBLE_DEVICES, enabling cuDF and cuGraph operations independent of LLM on GPU 0.

In [7]:
import cugraph
import pandas as pd

print("="*70)
print("📊 BUILDING KNOWLEDGE GRAPH")
print("="*70)

# Convert relationships to edge list
edges_data = []
nodes_set = set()

for rel in all_relationships:
    source = rel.get('source', '')
    target = rel.get('target', '')
    rel_type = rel.get('type', 'related_to')
    
    if source and target:
        edges_data.append({
            'source': source,
            'target': target,
            'type': rel_type
        })
        nodes_set.add(source)
        nodes_set.add(target)

# Create node mapping
nodes_list = sorted(list(nodes_set))
node_to_id = {node: i for i, node in enumerate(nodes_list)}

# Create cuDF edge list
edges_cudf = cudf.DataFrame({
    'source': [node_to_id[e['source']] for e in edges_data],
    'target': [node_to_id[e['target']] for e in edges_data],
    'type': [e['type'] for e in edges_data]
})

print(f"\n📊 Knowledge Graph:")
print(f"   Nodes (entities): {len(nodes_list)}")
print(f"   Edges (relationships): {len(edges_cudf)}")

# Create cuGraph
G = cugraph.Graph()
G.from_cudf_edgelist(edges_cudf, source='source', destination='target')

print(f"\n✅ Knowledge graph created on GPU 1")

📊 BUILDING KNOWLEDGE GRAPH

📊 Knowledge Graph:
   Nodes (entities): 14
   Edges (relationships): 11

✅ Knowledge graph created on GPU 1


## Step 8: GPU-Accelerated Graph Analytics

Computes graph metrics (centrality, communities, PageRank) on GPU 1 to identify key entities and relationship patterns in the knowledge graph.

Defines sample document corpus for knowledge extraction containing entities (people, organizations, places) and their relationships.

In [8]:
print("="*70)
print("🔬 GPU-ACCELERATED KNOWLEDGE GRAPH ANALYTICS")
print("="*70)

# PageRank - identify important entities
print("\n📊 PageRank Analysis (Entity Importance):")
pagerank = cugraph.pagerank(G)
pagerank = pagerank.sort_values('pagerank', ascending=False)

for _, row in pagerank.to_pandas().head(5).iterrows():
    node_id = int(row['vertex'])
    score = row['pagerank']
    entity = nodes_list[node_id] if node_id < len(nodes_list) else f"Node {node_id}"
    print(f"   {entity}: {score:.4f}")

# Betweenness Centrality - identify connector entities
print("\n📊 Betweenness Centrality (Connector Entities):")
bc = cugraph.betweenness_centrality(G)
bc = bc.sort_values('betweenness_centrality', ascending=False)

for _, row in bc.to_pandas().head(5).iterrows():
    node_id = int(row['vertex'])
    score = row['betweenness_centrality']
    entity = nodes_list[node_id] if node_id < len(nodes_list) else f"Node {node_id}"
    print(f"   {entity}: {score:.4f}")

print("\n✅ Graph analytics computed on GPU 1")

🔬 GPU-ACCELERATED KNOWLEDGE GRAPH ANALYTICS

📊 PageRank Analysis (Entity Importance):
   RAPIDS: 0.1182
   CUDA: 0.1141
   NCCL: 0.1042
   llama.cpp: 0.1042
   Graphistry: 0.0832

📊 Betweenness Centrality (Connector Entities):


/usr/local/lib/python3.12/dist-packages/cugraph/link_analysis/pagerank.py:232: UserWarning: Pagerank expects the 'store_transposed' flag to be set to 'True' for optimal performance during the graph creation
  warnings.warn(warning_msg, UserWarning)


   CUDA: 0.1923
   RAPIDS: 0.1410
   GPU: 0.1282
   Graphistry: 0.0769
   NCCL: 0.0128

✅ Graph analytics computed on GPU 1


## Step 9: LLM Analysis of Graph Insights

Computes graph metrics (centrality, communities, PageRank) on GPU 1 to identify key entities and relationship patterns in the knowledge graph.

Uses LLM on GPU 0 to extract named entities from documents, identifying people, organizations, locations, and other key concepts.

In [9]:
print("="*70)
print("🤔 LLM ANALYSIS OF GRAPH INSIGHTS")
print("="*70)

# Get top entities
top_pr = pagerank.to_pandas().head(3)
top_entities = [nodes_list[int(row['vertex'])] for _, row in top_pr.iterrows() if int(row['vertex']) < len(nodes_list)]

prompt = f"""I extracted a knowledge graph from documents about GPU computing. 
PageRank analysis shows the most important entities are: {', '.join(top_entities)}.

What does this tell us about the GPU computing ecosystem? Answer in 3-4 sentences."""

response = client.chat.create(
    messages=[{"role": "user", "content": prompt}],
    max_tokens=200,
    temperature=0.7
)

print(f"\n📋 LLM Analysis (GPU 0):")
print(response.choices[0].message.content)

print("\n✅ Simultaneous GPU operation:")
print("   GPU 0: LLM inference")
print("   GPU 1: Graph analytics (previously computed)")

🤔 LLM ANALYSIS OF GRAPH INSIGHTS

📋 LLM Analysis (GPU 0):
This PageRank analysis suggests that the top three entities in the extracted knowledge graph, RAPIDS, CUDA, and NCCL, hold significant importance in the GPU computing ecosystem. This implies that these three entities are closely interconnected and play crucial roles in facilitating or enabling GPU-based computations. RAPIDS is likely a tool or framework for GPU computing, while CUDA is a programming model developed by NVIDIA for high-performance computing on GPUs. NCCL may be a communication library used between GPUs.

✅ Simultaneous GPU operation:
   GPU 0: LLM inference
   GPU 1: Graph analytics (previously computed)


## Step 10: Graphistry Visualization

Extracts relationships between entities using LLM prompts designed for relationship identification (works_for, located_in, partners_with).

In [10]:
from kaggle_secrets import UserSecretsClient

user_secrets = UserSecretsClient()
graphistry.register(
    api=3,
    protocol="https",
    server="hub.graphistry.com",
    personal_key_id=user_secrets.get_secret("Graphistry_Personal_Key_ID"),
    personal_key_secret=user_secrets.get_secret("Graphistry_Personal_Secret_Key")
)


Authenticates with Graphistry cloud service using Kaggle secrets to enable interactive knowledge graph visualization and sharing.

Constructs knowledge graph on GPU 1 from extracted entities and relationships using cuDF DataFrames and cuGraph graph structures.

In [11]:
print("="*70)
print("🎨 ADVANCED GRAPHISTRY KNOWLEDGE GRAPH VISUALIZATION")
print("="*70)

import graphistry
from kaggle_secrets import UserSecretsClient
import pandas as pd
import numpy as np

# --- 1. Register Graphistry ---
print("\n🔐 Registering with Graphistry...")
try:
    user_secrets = UserSecretsClient()
    graphistry_key_id = user_secrets.get_secret("Graphistry_Personal_Key_ID")
    graphistry_secret = user_secrets.get_secret("Graphistry_Personal_Secret_Key")

    graphistry.register(
        api=3,
        protocol="https",
        server="hub.graphistry.com",
        personal_key_id=graphistry_key_id,
        personal_key_secret=graphistry_secret
    )
    print("✅ Graphistry registered successfully")
except Exception as e:
    print(f"⚠️ Graphistry registration failed: {e}")
    print("   Add secrets: Graphistry_Personal_Key_ID, Graphistry_Personal_Secret_Key")

# --- 2. Prepare Node DataFrame with Metrics ---
print("\n📊 Preparing knowledge graph data...")

# Convert cuDF to pandas
pagerank_pd = pagerank.to_pandas()
bc_pd = bc.to_pandas()

# Create edges DataFrame with entity names
edges_pd = pd.DataFrame({
    'source': [node_to_id[e['source']] for e in edges_data],
    'target': [node_to_id[e['target']] for e in edges_data],
    'type': [e['type'] for e in edges_data],
    'source_name': [e['source'] for e in edges_data],
    'target_name': [e['target'] for e in edges_data]
})

# Create nodes DataFrame with all metrics
nodes_pd = pd.DataFrame({
    'node_id': list(range(len(nodes_list))),
    'name': nodes_list
})

# Merge PageRank
nodes_pd = nodes_pd.merge(
    pagerank_pd.rename(columns={'vertex': 'node_id'}),
    on='node_id',
    how='left'
)

# Merge Betweenness Centrality
nodes_pd = nodes_pd.merge(
    bc_pd.rename(columns={'vertex': 'node_id'}),
    on='node_id',
    how='left'
)

# Compute degree centrality
degree_in = edges_pd.groupby('target').size().reset_index(name='degree_in').rename(columns={'target': 'node_id'})
degree_out = edges_pd.groupby('source').size().reset_index(name='degree_out').rename(columns={'source': 'node_id'})

nodes_pd = nodes_pd.merge(degree_in, on='node_id', how='left')
nodes_pd = nodes_pd.merge(degree_out, on='node_id', how='left')
nodes_pd['degree_in'] = nodes_pd['degree_in'].fillna(0)
nodes_pd['degree_out'] = nodes_pd['degree_out'].fillna(0)
nodes_pd['total_degree'] = nodes_pd['degree_in'] + nodes_pd['degree_out']

# --- 3. Louvain Community Detection ---
print("   Running Louvain community detection...")
try:
    import cugraph
    import cudf
    edges_cudf_temp = cudf.DataFrame({
        'source': edges_pd['source'].values,
        'destination': edges_pd['target'].values
    })
    G_temp = cugraph.Graph()
    G_temp.from_cudf_edgelist(edges_cudf_temp, source='source', destination='destination')

    parts = cugraph.louvain(G_temp)
    communities_pd = (parts[0] if isinstance(parts, tuple) else parts).to_pandas()
    communities_pd.columns = ['node_id', 'community']
    nodes_pd = nodes_pd.merge(communities_pd, on='node_id', how='left')
    nodes_pd['community'] = nodes_pd['community'].fillna(0).astype(int)
    print(f"   ✅ Found {nodes_pd['community'].nunique()} communities")
except Exception as e:
    print(f"   ⚠️ Fallback to degree-based communities: {e}")
    nodes_pd['community'] = pd.qcut(nodes_pd['total_degree'], q=min(3, len(nodes_pd)), labels=False, duplicates='drop')

# --- 4. Entity Type Classification ---
print("   Classifying entity types...")

def classify_entity_type(name):
    """Classify entities into types for knowledge graphs."""
    name_lower = name.lower()
    if any(x in name_lower for x in ['nvidia', 'graphistry', 'pytorch']):
        return 'Organization'
    elif any(x in name_lower for x in ['cuda', 'nccl', 'rapids', 'cudf', 'cugraph']):
        return 'Technology'
    elif any(x in name_lower for x in ['llama.cpp', 'gguf']):
        return 'Library'
    elif 'gpu' in name_lower:
        return 'Hardware'
    else:
        return 'Concept'

nodes_pd['entity_type'] = nodes_pd['name'].apply(classify_entity_type)

# --- 5. Role Classification based on Metrics ---
def classify_role(row):
    """Classify node roles: Hub (high centrality), Bridge (high betweenness), Peripheral."""
    pr = row['pagerank']
    bc = row['betweenness_centrality']
    pr_threshold = nodes_pd['pagerank'].median()
    bc_threshold = nodes_pd['betweenness_centrality'].median()

    if pr > pr_threshold and bc > bc_threshold:
        return 'Hub'
    elif bc > bc_threshold:
        return 'Bridge'
    else:
        return 'Peripheral'

nodes_pd['role'] = nodes_pd.apply(classify_role, axis=1)

# --- 6. Size Encoding ---
# Normalize PageRank to size range 25-100
pr_min = nodes_pd['pagerank'].min()
pr_max = nodes_pd['pagerank'].max()
nodes_pd['node_size'] = 25 + (nodes_pd['pagerank'] - pr_min) / (pr_max - pr_min) * 75

# --- 7. Create Rich Tooltips ---
nodes_pd['point_title'] = nodes_pd.apply(
    lambda row: f"{row['name']}\n" +
                f"Type: {row['entity_type']}\n" +
                f"Role: {row['role']}\n" +
                f"Community: {row['community']}\n" +
                f"PageRank: {row['pagerank']:.4f}\n" +
                f"Betweenness: {row['betweenness_centrality']:.4f}\n" +
                f"Degree: {int(row['total_degree'])}",
    axis=1
)

edges_pd['edge_title'] = edges_pd.apply(
    lambda row: f"{row['source_name']} → {row['target_name']}\n" +
                f"Relationship: {row['type']}",
    axis=1
)

print(f"\n📊 Knowledge Graph Summary:")
print(f"   Entities: {len(nodes_pd)}")
print(f"   Relationships: {len(edges_pd)}")
print(f"   Communities: {nodes_pd['community'].nunique()}")
print(f"   Entity Types: {nodes_pd['entity_type'].nunique()}")

# --- 8. Graphistry Visualization with Advanced Encodings ---
print("\n🎨 Creating sophisticated Graphistry visualization...")

# Color palettes
entity_type_colors = {
    'Organization': '#E74C3C',  # Red
    'Technology': '#3498DB',    # Blue
    'Library': '#2ECC71',       # Green
    'Hardware': '#F39C12',      # Orange
    'Concept': '#9B59B6'        # Purple
}

community_colors = {
    0: '#FF6B6B',
    1: '#4ECDC4',
    2: '#45B7D1',
    3: '#FFA07A',
    4: '#98D8C8'
}

role_icons = {
    'Hub': 'star',
    'Bridge': 'exchange-alt',
    'Peripheral': 'circle'
}

# Bind and create visualization
g = graphistry.bind(
    source='source',
    destination='target',
    node='node_id',
    point_title='point_title',
    edge_title='edge_title'
)

plotter = (
    g.edges(edges_pd)
    .nodes(nodes_pd)
    .encode_point_color('entity_type', categorical_mapping=entity_type_colors, default_mapping='#95A5A6')
    .encode_point_size('node_size')
    .encode_point_icon('role', categorical_mapping=role_icons, default_mapping='circle')
    .encode_point_badge('community', categorical_mapping=community_colors)
    .encode_edge_color('type', default_mapping='#BDC3C7')
    .settings(url_params={
        'play': 0,
        'pointSize': 2.5,
        'edgeOpacity': 0.6,
        'bg': '%23FFFFFF',
        'strongGravity': 'true',
        'edgeInfluence': 2.0
    })
)

# --- 9. Launch Visualization ---
try:
    url = plotter.plot(
        render=False,
        name="LLM Knowledge Graph - GPU Computing Ecosystem",
        description=f"Extracted {len(edges_pd)} relationships from {len(documents)} documents using llamatelemetry v0.1.0"
    )

    print(f"\n🚀 Visualization Created Successfully!")
    print(f"\n🔗 Graphistry URL:")
    print(f"   {url}")
    print(f"\n📌 Features:")
    print(f"   ✓ Color-coded by entity type (Organization, Technology, Library, etc.)")
    print(f"   ✓ Size scaled by PageRank importance")
    print(f"   ✓ Icons show role (Hub ⭐, Bridge ↔️, Peripheral ○)")
    print(f"   ✓ Community badges for clustering")
    print(f"   ✓ Interactive tooltips with full metrics")

    from IPython.display import display, HTML
    display(HTML(
        f'<div style="margin:20px; padding:20px; background:linear-gradient(135deg, #667eea 0%, #764ba2 100%); '
        f'border-radius:12px; color:white; box-shadow:0 4px 6px rgba(0,0,0,0.1);">'
        f'<h3 style="margin:0 0 10px 0;">🕸️ Knowledge Graph Dashboard</h3>'
        f'<p style="margin:5px 0;">Sophisticated GPU computing ecosystem visualization</p>'
        f'<a href="{url}" target="_blank" style="display:inline-block; margin-top:15px; padding:12px 24px; '
        f'background:white; color:#667eea; text-decoration:none; border-radius:6px; font-weight:bold; '
        f'box-shadow:0 2px 4px rgba(0,0,0,0.1);">🚀 Open Interactive Visualization</a>'
        f'</div>'
    ))

except Exception as e:
    print(f"\n❌ Visualization error: {e}")
    print(f"   Data prepared successfully - {len(nodes_pd)} nodes, {len(edges_pd)} edges")

print("\n" + "="*70)
print("🎯 Next: Advanced Graph Analytics (Step 10B)")
print("="*70)

🎨 ADVANCED GRAPHISTRY KNOWLEDGE GRAPH VISUALIZATION

🔐 Registering with Graphistry...
✅ Graphistry registered successfully

📊 Preparing knowledge graph data...
   Running Louvain community detection...
   ✅ Found 4 communities
   Classifying entity types...

📊 Knowledge Graph Summary:
   Entities: 14
   Relationships: 11
   Communities: 4
   Entity Types: 5

🎨 Creating sophisticated Graphistry visualization...

🚀 Visualization Created Successfully!

🔗 Graphistry URL:
   https://hub.graphistry.com/graph/graph.html?dataset=4d2ae70dad24432cb441beafe700738e&type=arrow&viztoken=d53e0f9d-e8da-4385-bb6a-5acc7b9afc3e&usertag=d0f479e4-pygraphistry-0.50.6&splashAfter=1770243052&info=true&play=0&pointSize=2.5&edgeOpacity=0.6&bg=%23FFFFFF&strongGravity=true&edgeInfluence=2.0

📌 Features:
   ✓ Color-coded by entity type (Organization, Technology, Library, etc.)
   ✓ Size scaled by PageRank importance
   ✓ Icons show role (Hub ⭐, Bridge ↔️, Peripheral ○)
   ✓ Community badges for clustering
   ✓ Int


🎯 Next: Advanced Graph Analytics (Step 10B)


## Step 10B: GPU-Accelerated Layouts & UMAP Clustering

Runs GPU-accelerated graph analytics (PageRank, centrality) on GPU 1 to identify most important entities and relationship patterns.

In [12]:
print("="*70)
print("⚡ GPU-ACCELERATED LAYOUT ALGORITHMS")
print("="*70)

# --- 1. GPU Force Atlas 2 Layout ---
print("\n🔥 Applying GPU Force Atlas 2 layout...")
print("   This layout uses GPU acceleration on GPU 1 via cuGraph")

try:
    # Use cuGraph-accelerated Force Atlas 2 layout
    plotter_gpu_layout = (
        plotter
        .layout_settings(
            play=8000,              # Auto-play animation for 8 seconds
            strongGravity=True,     # Pull nodes toward center
            edgeInfluence=2.0,      # Edge strength factor
            scalingRatio=2.0,       # Separation between nodes
            gravity=1.5,            # Center gravity strength
            barnesHutOptimize=True  # Barnes-Hut optimization
        )
    )
    
    url_fa2 = plotter_gpu_layout.plot(
        render=False,
        name="Knowledge Graph - GPU Force Atlas 2 Layout",
        description="GPU-accelerated force-directed layout optimized for knowledge graphs"
    )
    
    print(f"✅ Force Atlas 2 visualization created")
    print(f"   URL: {url_fa2}")
    print(f"   Features: Physics-based layout, community clustering")
    
    from IPython.display import display, HTML
    display(HTML(
        f'<div style="margin:15px 0; padding:15px; background:#e8f5e9; border-left:4px solid #4caf50; border-radius:6px;">'
        f'<strong>⚡ GPU Force Atlas 2 Layout</strong><br>'
        f'<a href="{url_fa2}" target="_blank" style="color:#2e7d32; font-weight:bold;">🚀 Open Visualization</a>'
        f'<p style="margin:5px 0 0 0; font-size:0.9em; color:#555;">Auto-plays for 8 seconds, then stabilizes</p>'
        f'</div>'
    ))
    
except Exception as e:
    print(f"⚠️ Force Atlas 2 layout error: {e}")

# --- 2. UMAP Dimensionality Reduction for Similarity Clustering ---
print("\n🗺️ Applying UMAP for entity similarity clustering...")
print("   UMAP creates edges between similar entities based on attributes")

try:
    # Create feature matrix for UMAP
    # Use entity metrics as features for similarity
    feature_cols = ['pagerank', 'betweenness_centrality', 'total_degree']
    
    # Ensure we have numeric data
    umap_features = nodes_pd[feature_cols].fillna(0)
    
    # Create UMAP similarity graph
    # This will create new edges based on feature similarity
    g_umap = graphistry.umap(
        nodes_pd,
        X=feature_cols,
        y='entity_type',  # Color by entity type
        n_neighbors=3,    # Smaller neighborhood for sparse graph
        min_dist=0.5,     # Separation between clusters
        spread=2.0,       # Cluster spread
        verbose=True
    )
    
    # Combine with original knowledge graph edges
    # The UMAP graph shows metric-based similarity
    # The original graph shows extracted relationships
    plotter_umap = (
        g_umap
        .bind(point_title='point_title')
        .encode_point_color('entity_type', categorical_mapping=entity_type_colors, default_mapping='#95A5A6')
        .encode_point_size('node_size')
        .encode_point_icon('role', categorical_mapping=role_icons)
        .settings(url_params={
            'play': 5000,
            'pointSize': 3.0,
            'edgeOpacity': 0.4,
            'bg': '%23F8F9FA'
        })
    )
    
    url_umap = plotter_umap.plot(
        render=False,
        name="Knowledge Graph - UMAP Similarity Clustering",
        description="Entity similarity based on PageRank, Betweenness, and Degree metrics"
    )
    
    print(f"✅ UMAP similarity visualization created")
    print(f"   URL: {url_umap}")
    print(f"   Features: Metric-based clustering, reveals hidden patterns")
    
    display(HTML(
        f'<div style="margin:15px 0; padding:15px; background:#e3f2fd; border-left:4px solid #2196f3; border-radius:6px;">'
        f'<strong>🗺️ UMAP Similarity Clustering</strong><br>'
        f'<a href="{url_umap}" target="_blank" style="color:#1565c0; font-weight:bold;">🚀 Open Visualization</a>'
        f'<p style="margin:5px 0 0 0; font-size:0.9em; color:#555;">Shows entities clustered by metric similarity (PageRank, Betweenness, Degree)</p>'
        f'</div>'
    ))
    
except Exception as e:
    print(f"⚠️ UMAP clustering error: {e}")
    print(f"   Note: UMAP works best with larger graphs (20+ nodes)")

# --- 3. Ring Layout by Community ---
print("\n⭕ Creating ring layout by community...")

try:
    # Ring layout groups nodes by community in circular arrangement
    plotter_ring = (
        plotter
        .encode_point_color('community', categorical_mapping=community_colors, default_mapping='#95A5A6')
        .settings(url_params={
            'play': 0,
            'layout': 'ring',
            'pointSize': 3.0,
            'edgeOpacity': 0.5,
            'bg': '%23FAFAFA'
        })
    )
    
    url_ring = plotter_ring.plot(
        render=False,
        name="Knowledge Graph - Ring Layout by Community",
        description="Circular layout grouped by Louvain communities"
    )
    
    print(f"✅ Ring layout visualization created")
    print(f"   URL: {url_ring}")
    print(f"   Features: Community-based circular arrangement")
    
    display(HTML(
        f'<div style="margin:15px 0; padding:15px; background:#fff3e0; border-left:4px solid #ff9800; border-radius:6px;">'
        f'<strong>⭕ Ring Layout by Community</strong><br>'
        f'<a href="{url_ring}" target="_blank" style="color:#e65100; font-weight:bold;">🚀 Open Visualization</a>'
        f'<p style="margin:5px 0 0 0; font-size:0.9em; color:#555;">Entities arranged in rings by community membership</p>'
        f'</div>'
    ))
    
except Exception as e:
    print(f"⚠️ Ring layout error: {e}")

print("\n" + "="*70)
print("💡 Layout Comparison:")
print("   • Force Atlas 2: Physics-based, reveals natural clustering")
print("   • UMAP: Metric similarity, finds hidden patterns")
print("   • Ring: Community-based, clear group visualization")
print("="*70)

⚡ GPU-ACCELERATED LAYOUT ALGORITHMS

🔥 Applying GPU Force Atlas 2 layout...
   This layout uses GPU acceleration on GPU 1 via cuGraph
⚠️ Force Atlas 2 layout error: PlotterBase.layout_settings() got an unexpected keyword argument 'strongGravity'

🗺️ Applying UMAP for entity similarity clustering...
   UMAP creates edges between similar entities based on attributes
⚠️ UMAP clustering error: module 'graphistry' has no attribute 'umap'
   Note: UMAP works best with larger graphs (20+ nodes)

⭕ Creating ring layout by community...
✅ Ring layout visualization created
   URL: https://hub.graphistry.com/graph/graph.html?dataset=ec7051b2fc434187a8d47cc503257866&type=arrow&viztoken=eecb1b46-469d-4929-8758-b93804f2572f&usertag=d0f479e4-pygraphistry-0.50.6&splashAfter=1770243130&info=true&play=0&pointSize=3.0&edgeOpacity=0.5&bg=%23FAFAFA&strongGravity=true&edgeInfluence=2.0&layout=ring
   Features: Community-based circular arrangement



💡 Layout Comparison:
   • Force Atlas 2: Physics-based, reveals natural clustering
   • UMAP: Metric similarity, finds hidden patterns
   • Ring: Community-based, clear group visualization


## Step 10C: GFQL Pattern Mining for Relationship Exploration

Computes community detection (Louvain) on knowledge graph to discover entity clusters and thematic groupings in the extracted knowledge.

In [13]:
print("="*70)
print("🔍 GFQL PATTERN MINING - Graph Query Language")
print("="*70)

print("\nGFQL enables dataframe-native graph pattern queries.")
print("Let's mine specific relationship patterns from the knowledge graph.\n")

# Import GFQL
from graphistry import n, e_forward, e_reverse, e_undirected

# --- Pattern 1: Technology Hubs (nodes with high connectivity) ---
print("🎯 Pattern 1: Finding Technology Hubs")
print("   Query: Nodes with 2+ outgoing edges (developers/enablers)")

try:
    # Create base graph for GFQL
    g_base = graphistry.bind(source='source', destination='target', node='node_id').edges(edges_pd).nodes(nodes_pd)
    
    # Pattern: Find nodes with multiple outgoing connections
    g_tech_hubs = g_base.gfql([
        n(name='tech_hub'),
        e_forward(hops=1),  # Outgoing edges
        n(name='target1'),
        e_forward(hops=1),  # Another outgoing from same source
        n(name='target2')
    ])
    
    # Get the filtered result
    hub_nodes = g_tech_hubs._nodes
    if hub_nodes is not None and len(hub_nodes) > 0:
        print(f"   ✅ Found {len(hub_nodes)} technology hub nodes")
        print(f"   Hubs: {', '.join(hub_nodes['name'].head(5).tolist())}")
    else:
        print("   ℹ️ No multi-connection hubs found (graph may be too sparse)")
    
except Exception as e:
    print(f"   ⚠️ Pattern 1 error: {e}")

# --- Pattern 2: 2-Hop Paths (indirect relationships) ---
print("\n🎯 Pattern 2: Finding 2-Hop Relationship Paths")
print("   Query: A → B → C paths showing indirect connections")

try:
    # Pattern: Find 2-hop paths
    g_2hop = g_base.gfql([
        n(name='source_entity'),
        e_forward(hops=1),
        n(name='intermediary'),
        e_forward(hops=1),
        n(name='destination_entity')
    ])
    
    paths_edges = g_2hop._edges
    if paths_edges is not None and len(paths_edges) > 0:
        print(f"   ✅ Found {len(paths_edges)} 2-hop relationship edges")
        
        # Show some example paths
        if 'source_name' in edges_pd.columns and 'target_name' in edges_pd.columns:
            sample_paths = edges_pd.head(3)
            for idx, row in sample_paths.iterrows():
                print(f"   Path: {row.get('source_name', 'N/A')} → ... → {row.get('target_name', 'N/A')}")
    else:
        print("   ℹ️ Limited 2-hop paths in current graph")
    
except Exception as e:
    print(f"   ⚠️ Pattern 2 error: {e}")

# --- Pattern 3: Relationship Type Filtering ---
print("\n🎯 Pattern 3: Filtering by Relationship Type")
print("   Query: Find all 'develops' or 'enables' relationships")

try:
    # Filter edges by relationship type
    dev_edges = edges_pd[edges_pd['type'].str.contains('develop|enable', case=False, na=False)]
    
    if len(dev_edges) > 0:
        g_dev = (
            graphistry.bind(source='source', destination='target').edges(dev_edges).nodes(nodes_pd).bind(node='node_id')
            .encode_point_color('entity_type', categorical_mapping=entity_type_colors)
            .encode_edge_color('type', categorical_mapping={
                'develops': '#E74C3C',
                'enables': '#27AE60'
            }, default_mapping='#95A5A6')
            .settings(url_params={'play': 0, 'pointSize': 3.0, 'bg': '%23FFFFFF'})
        )
        
        url_dev = g_dev.plot(
            render=False,
            name="Knowledge Graph - Development & Enablement Relationships",
            description="Filtered view showing only development and enablement relationships"
        )
        
        print(f"   ✅ Found {len(dev_edges)} development/enablement edges")
        print(f"   Visualization created: {url_dev}")
        
        from IPython.display import display, HTML
        display(HTML(
            f'<div style="margin:15px 0; padding:15px; background:#f3e5f5; border-left:4px solid #9c27b0; border-radius:6px;">'
            f'<strong>🔍 Filtered: Development & Enablement</strong><br>'
            f'<a href="{url_dev}" target="_blank" style="color:#6a1b9a; font-weight:bold;">🚀 Open Visualization</a>'
            f'<p style="margin:5px 0 0 0; font-size:0.9em; color:#555;">Shows only "develops" and "enables" relationships</p>'
            f'</div>'
        ))
    else:
        print("   ℹ️ No development/enablement relationships found")
    
except Exception as e:
    print(f"   ⚠️ Pattern 3 error: {e}")

# --- Pattern 4: Ego Network (neighborhood exploration) ---
print("\n🎯 Pattern 4: Ego Network Exploration")
print("   Query: Show immediate neighborhood of high PageRank entities")

try:
    # Get top entity by PageRank
    top_entity = nodes_pd.nlargest(1, 'pagerank').iloc[0]
    top_id = int(top_entity['node_id'])
    top_name = top_entity['name']
    
    print(f"   Exploring neighborhood of: {top_name} (PageRank: {top_entity['pagerank']:.4f})")
    
    # Filter edges connected to this entity
    ego_edges = edges_pd[
        (edges_pd['source'] == top_id) | 
        (edges_pd['target'] == top_id)
    ]
    
    # Get all connected nodes
    connected_ids = set(ego_edges['source'].tolist() + ego_edges['target'].tolist())
    ego_nodes = nodes_pd[nodes_pd['node_id'].isin(connected_ids)]
    
    if len(ego_edges) > 0:
        g_ego = (
            graphistry.bind(source='source', destination='target').edges(ego_edges).nodes(ego_nodes).bind(node='node_id')
            .encode_point_color('entity_type', categorical_mapping=entity_type_colors)
            .encode_point_size('node_size')
            .settings(url_params={'play': 0, 'pointSize': 4.0, 'bg': '%23F5F5F5'})
        )
        
        url_ego = g_ego.plot(
            render=False,
            name=f"Ego Network - {top_name}",
            description=f"Immediate neighborhood of {top_name} (highest PageRank entity)"
        )
        
        print(f"   ✅ Ego network: {len(ego_nodes)} connected entities, {len(ego_edges)} relationships")
        print(f"   Connected to: {', '.join(ego_nodes['name'].head(5).tolist())}")
        
        display(HTML(
            f'<div style="margin:15px 0; padding:15px; background:#fce4ec; border-left:4px solid #e91e63; border-radius:6px;">'
            f'<strong>🎯 Ego Network: {top_name}</strong><br>'
            f'<a href="{url_ego}" target="_blank" style="color:#c2185b; font-weight:bold;">🚀 Open Visualization</a>'
            f'<p style="margin:5px 0 0 0; font-size:0.9em; color:#555;">Immediate connections of the most influential entity</p>'
            f'</div>'
        ))
    else:
        print(f"   ℹ️ No connections found for {top_name}")
    
except Exception as e:
    print(f"   ⚠️ Pattern 4 error: {e}")

print("\n" + "="*70)
print("💡 GFQL enables powerful graph pattern mining:")
print("   • Chain patterns: n() → e() → n() for path queries")
print("   • Hop control: Find multi-step relationships")
print("   • Type filtering: Focus on specific relationship types")
print("   • Ego networks: Explore entity neighborhoods")
print("="*70)

🔍 GFQL PATTERN MINING - Graph Query Language

GFQL enables dataframe-native graph pattern queries.
Let's mine specific relationship patterns from the knowledge graph.

🎯 Pattern 1: Finding Technology Hubs
   Query: Nodes with 2+ outgoing edges (developers/enablers)
   ✅ Found 7 technology hub nodes
   Hubs: NVIDIA, PyTorch, RAPIDS, CUDA, NCCL

🎯 Pattern 2: Finding 2-Hop Relationship Paths
   Query: A → B → C paths showing indirect connections
   ✅ Found 5 2-hop relationship edges
   Path: NVIDIA → ... → CUDA
   Path: CUDA → ... → GPU
   Path: llama.cpp → ... → LLM

🎯 Pattern 3: Filtering by Relationship Type
   Query: Find all 'develops' or 'enables' relationships
   ✅ Found 2 development/enablement edges
   Visualization created: https://hub.graphistry.com/graph/graph.html?dataset=e039d9a8166647de988fdb326c583e5a&type=arrow&viztoken=0f499231-c217-48fc-8f86-437a3aa967b4&usertag=d0f479e4-pygraphistry-0.50.6&splashAfter=1770243186&info=true&play=0&pointSize=3.0&bg=%23FFFFFF



🎯 Pattern 4: Ego Network Exploration
   Query: Show immediate neighborhood of high PageRank entities
   Exploring neighborhood of: RAPIDS (PageRank: 0.1182)
   ✅ Ego network: 4 connected entities, 3 relationships
   Connected to: CUDA, RAPIDS, cuDF, cuGraph



💡 GFQL enables powerful graph pattern mining:
   • Chain patterns: n() → e() → n() for path queries
   • Hop control: Find multi-step relationships
   • Type filtering: Focus on specific relationship types
   • Ego networks: Explore entity neighborhoods


## Step 10D: Hypergraph Visualization for Multi-Entity Relationships

Creates interactive Graphistry visualization of knowledge graph with entities as nodes, relationships as edges, colored by entity type.

In [14]:
print("="*70)
print("🕸️ HYPERGRAPH VISUALIZATION - Multi-Entity Relationships")
print("="*70)

from IPython.display import display, HTML

url_spotlight = None
url_hg = None

print("\nHypergraphs reveal complex relationships involving 3+ entities.\nPerfect for knowledge graphs where multiple entities participate in relationships.\n")

# --- 1. Prepare Hypergraph Data ---
print("📊 Preparing hypergraph representation...")

try:
    # Create a dataframe with multi-entity events/relationships
    # For knowledge graphs, we can group entities by shared attributes
    
    # Group 1: Entities in same community
    hypergraph_data = []
    
    for community_id in nodes_pd['community'].unique():
        community_entities = nodes_pd[nodes_pd['community'] == community_id]
        if len(community_entities) >= 2:
            # Create a "community event" connecting all members
            for _, entity in community_entities.iterrows():
                hypergraph_data.append({
                    'event_id': f'community_{community_id}',
                    'event_type': 'Community Cluster',
                    'entity': entity['name'],
                    'entity_type': entity['entity_type'],
                    'role': entity['role']
                })
    
    # Group 2: Entities connected by same relationship type
    for rel_type in edges_pd['type'].unique():
        type_edges = edges_pd[edges_pd['type'] == rel_type]
        if len(type_edges) >= 2:
            # Get all entities involved in this relationship type
            involved_ids = set(type_edges['source'].tolist() + type_edges['target'].tolist())
            for entity_id in list(involved_ids)[:5]:  # Limit to 5 entities per hyperedge
                entity = nodes_pd[nodes_pd['node_id'] == entity_id]
                if not entity.empty:
                    entity = entity.iloc[0]
                    hypergraph_data.append({
                        'event_id': f'rel_type_{rel_type.replace(" ", "_")}',
                        'event_type': f'Relationship: {rel_type}',
                        'entity': entity['name'],
                        'entity_type': entity.get('entity_type', 'Unknown'),
                        'role': entity.get('role', 'Unknown')
                    })
    
    if len(hypergraph_data) > 0:
        hypergraph_df = pd.DataFrame(hypergraph_data)
        
        print(
            f"   ✅ Created hypergraph with {len(hypergraph_df)} entity-event connections\n"
            f"   Events: {hypergraph_df['event_id'].nunique()}\n"
            f"   Entities: {hypergraph_df['entity'].nunique()}"
        )
        
        # --- 2. Create Hypergraph Visualization ---
        print("\n🎨 Creating hypergraph visualization...")
        
        # Use Graphistry's hypergraph function (fix: remove entity_types argument)
        hg = graphistry.hypergraph(
            hypergraph_df,
            direct=True,
            opts={
                'CATEGORIES': {
                    'entity': ['entity'],
                    'event': ['event_id']
                }
            }
        )
        
        # Get the resulting graph
        hg_graph = hg['graph']
        
        # Enhance with encodings
        plotter_hg = (
            hg_graph
            .encode_point_color('type', categorical_mapping={
                'entity': '#3498DB',    # Blue for entities
                'event': '#E74C3C'      # Red for events (hyperedges)
            }, default_mapping='#95A5A6')
            .encode_point_size('degree', categorical_mapping={
                1: 20,
                2: 35,
                3: 50,
                4: 65
            }, default_mapping=40)
            .settings(url_params={
                'play': 6000,
                'pointSize': 2.5,
                'edgeOpacity': 0.4,
                'bg': '%23FAFAFA',
                'strongGravity': 'false'
            })
        )
        
        url_hg = plotter_hg.plot(
            render=False,
            name="Knowledge Graph - Hypergraph Multi-Entity Relationships",
            description="Hypergraph showing complex relationships involving multiple entities"
        )
        
        print(
            "   ✅ Hypergraph visualization created\n"
            "   Features:\n"
            "      • Blue nodes = Entities\n"
            "      • Red nodes = Events/Relationships (hyperedges)\n"
            "      • Reveals multi-entity patterns"
        )
        
        display(HTML(
            f'<div style="margin:20px 0; padding:20px; background:linear-gradient(135deg, #ff6b6b 0%, #f06595 100%); '
            f'border-radius:12px; color:white; box-shadow:0 4px 6px rgba(0,0,0,0.1);">'
            f'<h3 style="margin:0 0 10px 0;">🕸️ Hypergraph Visualization</h3>'
            f'<p style="margin:5px 0;">Complex multi-entity relationship analysis</p>'
            f'<a href="{url_hg}" target="_blank" style="display:inline-block; margin-top:15px; padding:12px 24px; '
            f'background:white; color:#f06595; text-decoration:none; border-radius:6px; font-weight:bold; '
            f'box-shadow:0 2px 4px rgba(0,0,0,0.1);">🚀 Open Hypergraph</a>'
            f'<p style="margin:10px 0 0 0; font-size:0.85em; opacity:0.9;">Shows entities grouped by communities and relationship types</p>'
            f'</div>'
        ))
        
        # --- 3. Hypergraph Statistics ---
        event_sizes = hypergraph_df.groupby('event_id')['entity'].count()
        print(
            "\n📈 Hypergraph Statistics:\n"
            f"   Average entities per hyperedge: {event_sizes.mean():.1f}\n"
            f"   Largest hyperedge: {event_sizes.max()} entities\n"
            f"   Total hyperedges: {len(event_sizes)}"
        )
        print(f"   Average entities per hyperedge: {event_sizes.mean():.1f}")
        print(f"   Largest hyperedge: {event_sizes.max()} entities")
        print(f"   Total hyperedges: {len(event_sizes)}")
        
        # Show sample hyperedges
        print(f"\n   Sample Hyperedges:")
        for event_id in hypergraph_df['event_id'].unique()[:3]:
            event_entities = hypergraph_df[hypergraph_df['event_id'] == event_id]
            event_type = event_entities.iloc[0]['event_type']
            entity_list = ', '.join(event_entities['entity'].tolist()[:5])
            if len(event_entities) > 5:
                entity_list += f" (+{len(event_entities)-5} more)"
            print(f"   • {event_type}: {entity_list}")
    
        # --- 4. Community spotlight view ---
        try:
            top_comms = nodes_pd['community'].value_counts().nlargest(3).index.tolist()
            spotlight_nodes = nodes_pd[nodes_pd['community'].isin(top_comms)]
            spotlight_edges = edges_pd[
                edges_pd['source'].isin(spotlight_nodes['node_id']) &
                edges_pd['target'].isin(spotlight_nodes['node_id'])
            ]
            if not spotlight_nodes.empty and not spotlight_edges.empty:
                spotlight_plotter = (
                    graphistry.bind(source='source', destination='target')
                    .edges(spotlight_edges)
                    .nodes(spotlight_nodes)
                    .encode_point_color(
                        'entity_type',
                        categorical_mapping=entity_type_colors,
                        default_mapping='#95A5A6'
                    )
                    .encode_point_icon('role', categorical_mapping=role_icons, default_mapping='circle')
                    .settings(
                        url_params={
                            'play': 3500,
                            'pointSize': 2.8,
                            'edgeOpacity': 0.45,
                            'bg': '%23F0F4FF',
                            'strongGravity': 'true'
                        }
                    )
                )
                url_spotlight = spotlight_plotter.plot(
                    render=False,
                    name="Community Spotlight - GPU Knowledge Graph",
                    description="High-degree nodes inside the most connected communities"
                )
                print(
                    "   ✅ Community spotlight visualization created\n"
                    f"   URL: {url_spotlight}"
                )
                display(HTML(
                    f'<div style="margin:20px 0; padding:18px; background:#f0f4ff; border-left:5px solid #5c7cfa; border-radius:8px;">'
                    f'<strong>🌐 Community Spotlight</strong><br>'
                    f'<span style="font-size:0.9em;">Illuminates the top communities with dense connectivity.</span><br>'
                    f'<a href="{url_spotlight}" target="_blank" style="display:inline-block; margin-top:10px; padding:10px 20px; '
                    f'background:#5c7cfa; color:white; text-decoration:none; border-radius:5px; font-weight:bold;">🚀 Open Spotlight</a>'
                    f'</div>'
                ))
            else:
                print("   ℹ️ Community spotlight view skipped (insufficient community density)")
        except Exception as e:
            print(f"   ⚠️ Community spotlight visualization error: {e}")

    else:
        print("   ℹ️ Not enough data to create meaningful hypergraph")
        print("      Hypergraphs work best with dense, multi-entity relationships")

except Exception as e:
    print(f"⚠️ Hypergraph creation error: {e}")
    import traceback
    traceback.print_exc()

dashboard_entries = [
    ("LLM Knowledge Graph", globals().get('url'), "LLM + GPU encoded knowledge graph", "#4c6ef5", "Overview"),
    ("GPU Force Atlas 2", globals().get('url_fa2'), "Force Atlas 2 layout emphasizing communities", "#16a34a", "Layout"),
    ("UMAP Similarity", globals().get('url_umap'), "Metric similarity clustering", "#2563eb", "Similarity"),
    ("Community Rings", globals().get('url_ring'), "Ring layout grouped by community", "#db7d11", "Community"),
    ("Filtered Develop/Enable", globals().get('url_dev'), "Development and enablement paths", "#a855f7", "GFQL"),
    ("Ego Network", globals().get('url_ego'), "Neighborhood of the most influential entity", "#d97706", "Ego"),
    ("Hypergraph Insight", globals().get('url_hg'), "Multi-entity relationships as hypernodes", "#db2777", "Hypergraph"),
    ("Community Spotlight", url_spotlight, "Top communities with dense entity interaction", "#6366f1", "Spotlight")
]
cards = []
for title, link, description, accent, tag in dashboard_entries:
    if not link:
        continue
    cards.append(
        f"<div style=\"flex:1 1 260px; min-width:260px; border:1px solid {accent}; border-radius:10px; padding:16px; box-shadow:0 12px 30px rgba(15,23,42,0.08); background:#ffffff;\">"
        f"<p style=\"margin:0 0 6px 0; font-size:0.8em; font-weight:600; color:{accent}; text-transform:uppercase;\">{tag}</p>"
        f"<h3 style=\"margin:0 0 8px 0;\">{title}</h3>"
        f"<p style=\"margin:0 0 12px 0; color:#475569; font-size:0.9em;\">{description}</p>"
        f"<a href=\"{link}\" target=\"_blank\" style=\"display:inline-flex; align-items:center; gap:6px; padding:8px 18px; background:{accent}; color:white; border-radius:8px; text-decoration:none; font-weight:600;\">Open View</a>"
        f"</div>"
    )
if cards:
    dashboard_html = '<div style="display:flex; flex-wrap:wrap; gap:16px; margin-top:16px;">' + ''.join(cards) + '</div>'
    display(HTML(
        f'<section style="margin:25px 0; padding:20px 24px; background:#ffffff; border-radius:14px; box-shadow:0 16px 40px rgba(15,23,42,0.15);">'
        f'<h2 style="margin:0 0 10px 0;">Graphistry Multi-View Dashboard</h2>'
        f'<p style="margin:0 0 16px 0; color:#475569;">Launch the curated set of analytics views in a single dashboard panel.</p>'
        f'{dashboard_html}'
        f'</section>'
    ))
    print("   📊 Graphistry dashboard assembled with multiple curated views.")
else:
    print("   ℹ️ Dashboard not rendered (no visualization URLs are ready yet).")

print("\n" + "="*70)
print(
    "💡 Hypergraph Benefits:\n"
    "   • Represents relationships involving 3+ entities simultaneously\n"
    "   • Reveals community structures and shared contexts\n"
    "   • Goes beyond binary (A→B) relationships\n"
    "   • Useful for event-based knowledge graphs"
)
print("="*70)

print("\n" + "="*70)
print("🎉 ADVANCED GRAPHISTRY VISUALIZATION COMPLETE!")
print("="*70)
print(
    "\nYou now have multiple sophisticated views:\n"
    "   1. Step 10:  Multi-encoding dashboard (color, size, icon, badge)\n"
    "   2. Step 10B: GPU layouts (Force Atlas 2, UMAP, Ring)\n"
    "   3. Step 10C: GFQL pattern mining (hubs, paths, filters, ego)\n"
    "   4. Step 10D: Hypergraph (multi-entity relationships)"
)
print(
    "\nAll visualizations leverage:\n"
    "   ✓ GPU acceleration (cuGraph on GPU 1)\n"
    "   ✓ LLM-extracted knowledge (llama-server on GPU 0)\n"
    "   ✓ Interactive exploration (Graphistry cloud rendering)"
)
print("="*70)

🕸️ HYPERGRAPH VISUALIZATION - Multi-Entity Relationships

Hypergraphs reveal complex relationships involving 3+ entities.
Perfect for knowledge graphs where multiple entities participate in relationships.

📊 Preparing hypergraph representation...
   ✅ Created hypergraph with 24 entity-event connections
   Events: 6
   Entities: 14

🎨 Creating hypergraph visualization...
# links 240
# events 24
# attrib entities 30


/usr/local/lib/python3.12/dist-packages/graphistry/util.py:241: RuntimeWarning: Encoding validation warning: {'message': 'Field node_encodings.complex.default.pointSizeEncoding.attribute does not exist in node.attributes', 'data': {'attribute': 'degree', 'node.attributes': "['event_id', 'nodeTitle', 'type', 'category', 'nodeID', 'event_type', 'entity', 'entity_type', 'role', 'EventID']"}}
  warnings.warn(RuntimeWarning(msg))


   ✅ Hypergraph visualization created
   Features:
      • Blue nodes = Entities
      • Red nodes = Events/Relationships (hyperedges)
      • Reveals multi-entity patterns



📈 Hypergraph Statistics:
   Average entities per hyperedge: 4.0
   Largest hyperedge: 5 entities
   Total hyperedges: 6
   Average entities per hyperedge: 4.0
   Largest hyperedge: 5 entities
   Total hyperedges: 6

   Sample Hyperedges:
   • Community Cluster: CUDA, GPU, Graphistry, NVIDIA, RAPIDS cuGraph
   • Community Cluster: GGUF, LLM, llama.cpp
   • Community Cluster: NCCL, PyTorch, multi-GPU communication primitives
   ⚠️ Community spotlight visualization error: Node identifier must be bound when using node dataframe.


   📊 Graphistry dashboard assembled with multiple curated views.

💡 Hypergraph Benefits:
   • Represents relationships involving 3+ entities simultaneously
   • Reveals community structures and shared contexts
   • Goes beyond binary (A→B) relationships
   • Useful for event-based knowledge graphs

🎉 ADVANCED GRAPHISTRY VISUALIZATION COMPLETE!

You now have multiple sophisticated views:
   1. Step 10:  Multi-encoding dashboard (color, size, icon, badge)
   2. Step 10B: GPU layouts (Force Atlas 2, UMAP, Ring)
   3. Step 10C: GFQL pattern mining (hubs, paths, filters, ego)
   4. Step 10D: Hypergraph (multi-entity relationships)

All visualizations leverage:
   ✓ GPU acceleration (cuGraph on GPU 1)
   ✓ LLM-extracted knowledge (llama-server on GPU 0)
   ✓ Interactive exploration (Graphistry cloud rendering)


## Step 11: Entity Deep Dive with LLM

Demonstrates querying knowledge graph using LLM natural language understanding combined with cuGraph path finding and subgraph extraction.

In [16]:
print("="*70)
print("🔍 ENTITY DEEP DIVE")
print("="*70)

def analyze_entity(entity_name):
    """Use LLM to analyze a specific entity's role in the knowledge graph."""
    entity_data = nodes_pd[nodes_pd['name'] == entity_name]
    
    if entity_data.empty:
        return f"Entity '{entity_name}' not found."
    
    entity_data = entity_data.iloc[0]
    
    prompt = f"""Analyze the role of {entity_name} in the GPU computing ecosystem:
    - PageRank score: {entity_data.get('pagerank', 0):.4f} (importance)
    - Betweenness: {entity_data.get('betweenness_centrality', 0):.4f} (connectivity)
    
    Explain its significance in 2 sentences."""
    
    response = client.chat.create(
        messages=[{"role": "user", "content": prompt}],
        max_tokens=100,
        temperature=0.7
    )
    
    return response.choices[0].message.content

# Analyze top 3 entities
print("\n🔍 Entity Analysis:")
for entity in top_entities[:3]:
    print(f"\n📌 {entity}:")
    analysis = analyze_entity(entity)
    print(f"   {analysis}")

🔍 ENTITY DEEP DIVE

🔍 Entity Analysis:

📌 RAPIDS:
   I couldn't find any information about "RAPIDS" being a part of the PageRank or Betweenness centrality analysis, as these metrics are typically associated with network analysis and graph theory.

However, I found that RAPIDS (Rapidly Exploring Potential Solution Space) is a software framework for GPU-accelerated computing. It's primarily used for scientific computing, machine learning, and data analytics on NVIDIA GPUs.

In the context of the GPU computing ecosystem, RAPIDS plays a

📌 CUDA:
   Based on the provided PageRank and Betweenness scores, I can analyze the role of CUDA in the GPU computing ecosystem.

CUDA plays a pivotal role in the GPU computing ecosystem, enabling developers to harness the massive parallel processing capabilities of NVIDIA's graphics processing units (GPUs) to accelerate compute-intensive tasks such as scientific simulations, machine learning, and data analytics. With its significance highlighted by the hi

## Step 12: Monitor Both GPUs

Computes graph metrics (centrality, communities, PageRank) on GPU 1 to identify key entities and relationship patterns in the knowledge graph.

Uses LLM to generate natural language summaries of graph structure, key entities, and discovered relationship patterns.

In [17]:
print("="*70)
print("📊 DUAL GPU MONITORING")
print("="*70)

!nvidia-smi

print("\n💡 Split-GPU Operation:")
print("   GPU 0: llama-server (GGUF model for entity extraction)")
print("   GPU 1: RAPIDS (cuDF/cuGraph for knowledge graph analytics)")

📊 DUAL GPU MONITORING
Wed Feb  4 22:26:19 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 580.105.08             Driver Version: 580.105.08     CUDA Version: 13.0     |
+-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   48C    P0             26W /   70W |    2677MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-------------------------

## Step 13: Cleanup

Verifies GPU environment and ensures dual T4 setup is ready for knowledge graph extraction combining LLM-powered entity extraction with GPU-accelerated graph analytics.

Stops llama-server, releases RAPIDS GPU memory, and verifies both GPUs have freed all resources.

In [18]:
print("🛑 Stopping llama-server...")
server.stop_server()

# Clear RAPIDS memory
import gc
del G, edges_cudf, pagerank, bc
gc.collect()

print("\n✅ Resources cleaned up")
print("\n📊 Final GPU Status:")
!nvidia-smi --query-gpu=index,memory.used,memory.free --format=csv

🛑 Stopping llama-server...

✅ Resources cleaned up

📊 Final GPU Status:
index, memory.used [MiB], memory.free [MiB]
0, 111 MiB, 14802 MiB
1, 3 MiB, 14910 MiB


## 📚 Summary

### LLM-Powered Knowledge Graph Workflow:
1. **GPU 0**: LLM extracts entities & relationships from text
2. **GPU 1**: cuGraph builds and analyzes knowledge graph
3. **GPU 0**: LLM interprets graph analytics insights
4. **Graphistry**: Interactive visualization with metrics

### Key Integration Points:
- ✅ LLM for entity extraction (structured JSON output)
- ✅ cuGraph for GPU-accelerated graph algorithms
- ✅ Graphistry for interactive knowledge graph visualization
- ✅ LLM for semantic interpretation of graph metrics

### Split-GPU Architecture:
```python
# GPU 0: llama-server
tensor_split="1.0,0.0"  # 100% on GPU 0

# GPU 1: RAPIDS
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
import cudf, cugraph  # Uses GPU 1
```

---

**Next:** [08-document-network-analysis](08-document-network-analysis-graphistry-llamatelemetry-v0-1-0.ipynb)